In [13]:
from _lammps.get_thermo_from_log import get_last_structure_info_series_from_thermo_csv
from utils.constants.constants import Constants
import glob
import pandas as pd
import matplotlib.pyplot as plt

root_dir = '/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04'
structures = Constants.structures()

In [14]:
# create all lattice csv
for i, structure in enumerate(structures):
    dirs = glob.glob(f'{root_dir}/{structure}/*')
    for j, _dir in enumerate(dirs):
        print(_dir)
        if i==0 and j==0:
            df = get_last_structure_info_series_from_thermo_csv(dirs[0])
            continue
        tmp_df = get_last_structure_info_series_from_thermo_csv(_dir)
        df = pd.concat([df,tmp_df])
df.to_csv(f'{root_dir}/last_structure_info.csv', index=False)

/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_46.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_666.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_840.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_894.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_1056.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_446.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_132.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobalite/data_alpha-critobalite_278.xsf
/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04/alpha-critobal

In [77]:
from descriptors.base_info import get_reindex_base

base_df = get_reindex_base(is_gpu=False)
root_dir = '/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization'

def get_dft_compared_df(root_dir, base_df, df):
    def get_min_energy_structure_record(structure, original_df):
        df = original_df.copy()
        structure_df = df[df['last structure'] == structure]
        min_e = structure_df.Eng_a.min()
        record = structure_df[structure_df.Eng_a == min_e]
        if len(record) > 1:
            record = record[0:1]
        return record
    record = get_min_energy_structure_record(structures[0], df)
    for structure in structures[1:]:
        tmp_record = get_min_energy_structure_record(structure, df)
        record = pd.concat([record, tmp_record])

    record['is_DFT'] = 0
    base_df['Eng_a'] = base_df.E / base_df.natom
    dft_min_structure = base_df.groupby('structure',as_index=False).min()
    for structure in structures:
        print(structure)
        nn_energy = record[record['last structure'] == structure].Eng_a
        if len(nn_energy) > 0:
            nn_energy = nn_energy.iloc[-1]
        else:
            print(f'not exist {structure}')
            continue
        dft_energy = dft_min_structure[dft_min_structure.structure == structure].Eng_a.iloc[-1]
        if nn_energy < dft_energy:
            print(f'energy from nn({nn_energy}) is smaller than energy from dft({dft_energy})')
        elif nn_energy > dft_energy:
            print(f'energy from nn({nn_energy}) is bigger than energy from dft({dft_energy})')
            record.loc[record['last structure'] == structure, ['is_DFT']] = 1
        else:
            print('something wrong')

    return record

alpha-critobalite 1 1150
alpha-quartz 1151 2250
beta-quartz 2251 3050
beta-trydymite 3051 4300
coesite 4301 5500
Fdd2-beta-critobalite 5501 6850
hex-trydymite 6851 7850
stishovite 7851 8550


In [79]:
sheet_names = ['01', '02', '03', '04', '05', '06', '07']
with pd.ExcelWriter(f'{root_dir}/dft_compare_result.xlsx') as writer:
    for sheet_name in sheet_names:
        df = pd.read_excel(f'{root_dir}/select_structure.xlsx', sheet_name=f'{sheet_name}')
        record = get_dft_compared_df(root_dir, base_df, df)
        record.to_excel(writer, sheet_name=f'{sheet_name}')

alpha-critobalite
energy from nn(-573.421) is smaller than energy from dft(-573.4207506071069)
alpha-quartz
energy from nn(-573.4316) is bigger than energy from dft(-573.4317867043701)
beta-quartz
energy from nn(-573.4202) is smaller than energy from dft(-573.4201227063197)
beta-trydymite
energy from nn(-573.4213) is smaller than energy from dft(-573.4194774367224)
coesite
energy from nn(-573.4275) is bigger than energy from dft(-573.427720351426)
Fdd2-beta-critobalite
energy from nn(-573.4216) is smaller than energy from dft(-573.4211688633321)
hex-trydymite
not exist hex-trydymite
stishovite
energy from nn(-573.4343) is smaller than energy from dft(-573.4338082128206)
alpha-critobalite
energy from nn(-573.4227) is smaller than energy from dft(-573.4207506071069)
alpha-quartz
energy from nn(-573.433) is smaller than energy from dft(-573.4317867043701)
beta-quartz
energy from nn(-573.4213) is smaller than energy from dft(-573.4201227063197)
beta-trydymite
energy from nn(-573.4357) is s

,structure,idx,natom,TotEng,Temp,Lx,Volume,Eng_a,last structure,is_DFT
6,alpha-critobalite,132.0,48.0,-27524.230,0.0,9.932049,673.45137,-573.4215,alpha-critobalite,0
12,alpha-quartz,326.0,36.0,-20643.545,0.0,9.758075,443.21180,-573.4318,alpha-quartz,0
21,beta-quartz,2.0,36.0,-20643.141,0.0,10.111269,487.90483,-573.4206,beta-quartz,0
33,beta-trydymite,567.0,48.0,-27524.182,0.0,10.095901,698.72486,-573.4205,beta-trydymite,0
45,coesite,457.0,24.0,-13762.265,0.0,7.122698,271.09392,-573.4277,coesite,1
50,Fdd2-beta-critobalite,108.0,48.0,-27524.234,0.0,10.104371,702.82095,-573.4215,Fdd2-beta-critobalite,0
69,hex-trydymite,46.0,48.0,-27523.646,0.0,10.438013,803.49211,-573.4093,hex-trydymite,0
72,stishovite,481.0,48.0,-27524.940,0.0,8.278472,369.06861,-573.4362,stishovite,0
